In [9]:
import os
import pandas as pd
import boto3

from sagemaker import ModelPackage
import sagemaker as sagemaker
from sagemaker import get_execution_role


from src.configuration import Configuration
from src.services import S3, Woocommerce

In [10]:
config = Configuration('configs/config.json').parameters

In [11]:
# Configure access to AWS and WordPress

s3 = S3(region_name = config['s3']['region_name'],
        access_key =  config['aws_keys']["access_key"],
       secret_key = config['aws_keys']["secret_key"],
       bucket = config['s3']["bucket"])

woocommerce = Woocommerce(url=config['woocommerce']["url"],
                         consumer_key=config['woocommerce']["consumer_key"],
                         consumer_secret=config['woocommerce']["consumer_secret"])

In [4]:
# Communicate with the MySQL WordPress database via the Woocommerce Rest API

woocommerce.download_data('data/input_data.csv')

In [12]:
# Upload transactional data to S3

s3.upload('data/input_data.csv','input_data.csv')


model_package_arn = config['batch_transform_job']['model_package_arn']

iam = boto3.client('iam')
role = iam.get_role(RoleName=config['batch_transform_job']['role_name'])['Role']['Arn']

sagemaker_session = sagemaker.Session()


model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)




output_path = 's3://' + os.path.join(config['s3']['bucket'],'output')

transformer = model.transformer(instance_count=config['batch_transform_job']['instance_count'], 
                                instance_type=config['batch_transform_job']['instance_type'],
                                output_path=output_path)



input_path = 's3://' + os.path.join(config["s3"]["bucket"], config["s3"]["input_file"])


transformer.transform(input_path, content_type='text/csv')

transformer.wait()

.

KeyboardInterrupt: 

In [13]:
# Recommendations produced by the model are stored in S3. Donwloading them:


bucket_folder = transformer.output_path.rsplit('/')[3]
key = os.path.join(bucket_folder, config["s3"]["input_file"].split('/')[-1]+'.out')


s3.download(key, 'data/output_data.csv')

In [14]:
# loading and visualizing the output recommendations

output_df  = pd.read_csv('data/output_data.csv')
output_df  = output_df.drop('Unnamed: 0',1)
out_final = output_df
out_final.head()

,Item in cart,Recommendation,Item Support,Support,Confidence,Lift,Leverage,Conviction
0,RED ORCHID,GREEN ORCHID,0.249443,0.133630,0.535714,3.340774,0.093630,1.808463
1,RED ORCHID,BONSAI PINE,0.249443,0.075724,0.303571,2.163549,0.040724,1.234424
2,RED ORCHID,SWEET WILLIAM,0.249443,0.071269,0.285714,2.729483,0.045159,1.253452
3,RED ORCHID,WHITE ORCHID,0.249443,0.064588,0.258929,2.004464,0.032366,1.175088
4,RED ORCHID,SUNFLOWER,0.249443,0.057906,0.232143,1.966644,0.028462,1.148599


In [15]:
## We push them to our online shop using the Woocommerce Rest API

woocommerce.upload_recommendations('data/output_data.csv')